### Import:

In [ ]:
import pandas as pd
import numpy as np
import os
import time

### Caricamento del dataset:

In [ ]:
try:
    df = pd.read_csv("./data/monsters.csv")
except FileNotFoundError:
    print("Errore: Il file './data/monsters.csv' non esiste. Controlla il percorso.")
    # Interrompiamo l'esecuzione se manca il file
    raise

## --- Conversione con NumPy (Vectorized) ---
### 1. Preparazione Dati
Gestiamo i valori nulli e convertiamo in stringhe minuscole per l'analisi.

In [ ]:
# Riempiamo i NaN con 'unaligned' e convertiamo in stringa minuscola
df['alignment_clean'] = df['alignment'].fillna('unaligned').astype(str).str.lower()

### 2. Definizione Condizioni
Creiamo le maschere booleane.

In [ ]:
# Condizioni per Ethics (Legge/Caos)
cond_lawful = df['alignment_clean'].str.contains('lawful')
cond_chaotic = df['alignment_clean'].str.contains('chaotic')

# Condizioni per Morale (Bene/Male)
cond_good = df['alignment_clean'].str.contains('good')
cond_evil = df['alignment_clean'].str.contains('evil')

### 3. Calcolo Punteggi (np.select)
Rinominiamo 'etica' in 'ethics'.

In [ ]:
# Calcolo Ethics: Lawful = 1, Chaotic = -1, Neutral = 0
df['ethics'] = np.select(
    [cond_lawful, cond_chaotic], 
    [1, -1], 
    default=0
)

# Calcolo Morale: Good = 1, Evil = -1, Neutral = 0
df['morale'] = np.select(
    [cond_good, cond_evil], 
    [1, -1], 
    default=0
)

### Pulizia
Rimuoviamo le colonne di supporto.

In [ ]:
cols_to_drop = ['alignment', 'alignment_clean']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
print("Colonna 'alignment' rimossa e convertita in numeri.")

## --- 2. RIORDINAMENTO COLONNE ---
L'obiettivo è avere: ..., sub_race, ethics, morale, ...

In [ ]:
cols = df.columns.tolist()

# Verifichiamo che tutte le colonne necessarie esistano (usando 'ethics')
if 'sub_race' in cols and 'ethics' in cols and 'morale' in cols:
    # Rimuoviamo le nuove colonne dalla loro posizione attuale
    cols.remove('ethics')
    cols.remove('morale')

    # Troviamo l'indice di sub_race
    target_index = cols.index('sub_race')

    # Inseriamo le colonne subito dopo (target_index + 1 e + 2)
    cols.insert(target_index + 1, 'ethics')
    cols.insert(target_index + 2, 'morale')

    # Applichiamo il riordinamento
    df = df[cols]
    print("Colonne riordinate: 'ethics' e 'morale' posizionate dopo 'sub_race'.")
else:
    print("ATTENZIONE: Impossibile riordinare. Controlla che 'sub_race', 'ethics' e 'morale' esistano.")

## --- 3. VERIFICA E SALVATAGGIO ---
Gestione PermissionError

In [ ]:
print("\n--- Anteprima Nuove Colonne ---")
print(df[['name', 'sub_race', 'ethics', 'morale']].head(10))

# Genera un nome file unico basato sull'orario attuale
timestamp = time.strftime("%H-%M-%S")
output_path = f"./data/monsters_{timestamp}.csv"

try:
    df.to_csv(output_path, index=False)
    print(f"✅ Successo! Dati salvati in: {output_path}")

    try:
        if os.path.exists("./data/monsters.csv"):
            os.remove("./data/monsters.csv")
            os.rename(output_path, "./data/monsters.csv")
            print("File sovrascritto con successo.")
        else:
            os.rename(output_path, "./data/monsters.csv")
            print("File rinominato correttamente in 'monsters.csv'")
    except PermissionError:
        print(f"⚠️ impossibile a sovrascrivere 'monsters.csv' perché è bloccato, ma i tuoi dati sono al sicuro nel file col timestamp!")

except Exception as e:
    print(f"❌ Errore critico: {e}")